> 跑起来，跑得对，跑得快；

- https://verl.readthedocs.io/en/latest/examples/config.html
- https://verl.readthedocs.io/en/latest/perf/perf_tuning.html
- https://verl.readthedocs.io/en/latest/perf/device_tuning.html
    - https://github.com/volcengine/verl/blob/main/examples/tuning/7b/qwen2-7b_grpo_2_h800_fsdp_vllm.sh
    - https://github.com/volcengine/verl/blob/main/examples/tuning/14b/qwen2_14b_grpo_4_h800_fsdp_vllm.sh
- https://github.com/volcengine/verl/blob/main/examples/grpo_trainer/run_qwen2-7b.sh
----
- main_ppo.py
    - 实例化 `trainer = RayPPOTrainer`，
    - `trainer.fit`
- ray_trainer.py 定义 generation/training 的 workflow/pipeline（任务调度）
    - generation (experience preparation)
        - generate_sequences
            - `ray::WorkerDict.actor_rollout_generate_sequences`
        - compute_log_prob
        - compute_ref_log_prob
        - reward_fn
        - advantage
    - training
        - update_actor
- ppo_trainer.py

### PPO & GRPO

$$
\mathcal{J}_{PPO}(\theta) = \mathbb{E}[q \sim P(Q), o \sim \pi_{\theta_{old}}(O|q)] \frac{1}{|o|} \sum_{t=1}^{|o|} \min \left[ \frac{\pi_{\theta}(o_t|q, o_{<t})}{\pi_{\theta_{old}}(o_t|q, o_{<t})} A_t, \text{clip} \left( \frac{\pi_{\theta}(o_t|q, o_{<t})}{\pi_{\theta_{old}}(o_t|q, o_{<t})}, 1 - \epsilon, 1 + \epsilon \right) A_t \right]
$$
- $r$ 的计算 （定义在token级别，**(reverse) kl term  within the reward**）
    - $r_t = r_{\phi}(q, o_{\le t}) - \beta \log \frac{\pi_{\theta}(o_t|q, o_{<t})}{\pi_{ref}(o_t|q, o_{<t})}$
- GAE（advantage）的计算(r, v => GAE)
    - $\delta_t = r_t + \gamma V(s_{t+1}) - V(s_t)$
    - $\hat{A}_t^{GAE(\gamma, \lambda)} = \sum_{l=0}^{\infty} (\gamma \lambda)^l \delta_{t+l}$
    - $\hat{A}_t^{GAE(\gamma, \lambda)} = \sum_{l=0}^{\infty} (\gamma \lambda)^l (r_{t+l} + \gamma V(s_{t+l+1}) - V(s_{t+l}))$
        - $\hat{A}_t^{GAE(\gamma, \lambda)}$: 在时间步 t 的广义优势估计。
        - $\gamma$: 折扣因子 (discount factor)，通常取值在 0 到 1 之间，表示未来奖励的重要性。
        - $\lambda$: GAE 参数，通常取值在 0 到 1 之间，用于在偏差 (bias) 和方差 (variance) 之间进行权衡。
            - 当 $\lambda = 0$ 时，GAE 退化为标准的 TD 优势估计：$\hat{A}_t = \delta_t = r_t + \gamma V(s_{t+1}) - V(s_t)$ (低偏差，高方差)。
            - 当 $\lambda = 1$ 时，GAE 考虑了直到回合结束的所有 TD 残差的折扣和，类似于蒙特卡洛优势估计 (高偏差，低方差)。

$$
\mathcal{J}_{GRPO}(\theta) = \mathbb{E}[q \sim P(Q), \{o_i\}_{i=1}^G \sim \pi_{\theta_{old}}(O|q)] \\
\frac{1}{G} \sum_{i=1}^G \frac{1}{|o_i|} \sum_{t=1}^{|o_i|} \left\{ \min \left[ \frac{\pi_{\theta}(o_{i,t}|q, o_{i,<t})}{\pi_{\theta_{old}}(o_{i,t}|q, o_{i,<t})} \hat{A}_{i,t}, \text{clip} \left( \frac{\pi_{\theta}(o_{i,t}|q, o_{i,<t})}{\pi_{\theta_{old}}(o_{i,t}|q, o_{i,<t})}, 1 - \epsilon, 1 + \epsilon \right) \hat{A}_{i,t} \right] - \beta \mathbb{D}_{KL}[\pi_{\theta}||\pi_{ref}] \right\}
$$
- advantage
    - $\hat{A}_{i,t} = \tilde{r}_i = \frac{r_i - \text{mean}(r)}{\text{std}(r)}$
- kl estimation (**(reverse) kl term within loss**)
    - $\mathbb{D}_{KL}[\pi_\theta || \pi_{ref}] = \frac{\pi_{ref}(o_{i,t}|q, o_{i,<t})}{\pi_\theta(o_{i,t}|q, o_{i,<t})} - \log \frac{\pi_{ref}(o_{i,t}|q, o_{i,<t})}{\pi_\theta(o_{i,t}|q, o_{i,<t})} - 1$
    - $\mathbb{D}_{KL}[\pi_\theta || \pi_{ref}] = \sum_{y} \pi_\theta(y|q) \log \frac{\pi_\theta(y|q)}{\pi_{ref}(y|q)} = \mathbb{E}_{y \sim \pi_\theta(\cdot|q)} \left[ \sum_{t=1}^{T} \log \frac{\pi_\theta(o_t | q, o_{<t})}{\pi_{ref}(o_t | q, o_{<t})} \right]$
        - $\pi(y|q) = \pi(o_1, ..., o_T | q) = \prod_{t=1}^{T} \pi(o_t | q, o_{<t})$
        - $\log \frac{\pi_\theta(y|q)}{\pi_{ref}(y|q)} = \log \frac{\prod_{t=1}^{T} \pi_\theta(o_t | q, o_{<t})}{\prod_{t=1}^{T} \pi_{ref}(o_t | q, o_{<t})}$
            - $= \sum_{t=1}^{T} \log \pi_\theta(o_t | q, o_{<t}) - \sum_{t=1}^{T} \log \pi_{ref}(o_t | q, o_{<t})$
            - $= \sum_{t=1}^{T} \left[ \log \pi_\theta(o_t | q, o_{<t}) - \log \pi_{ref}(o_t | q, o_{<t}) \right]$
            - $= \sum_{t=1}^{T} \log \frac{\pi_\theta(o_t | q, o_{<t})}{\pi_{ref}(o_t | q, o_{<t})}$

- `actor.kl_loss_coef`：默认 0.001（`ppo_trainer.yaml`）
    - GRPO (`use_kl_loss` enable)
    - `kl_loss_type`: `low_var_kl`
        - k3 estimation
- `algorithm.kl_penalty` (=> `algorithm.use_kl_in_reward`)
    - in-reward kl penalty. 

### batch_size

> 
>    - Algorithmic metrics (train batch size, PPO mini-batch size) are **global** (from a single-controller perspective), normalized in each worker. See the normalization code.

>    - Performance-related parameters (**micro batch size**, max token length for dynamic batch size) are local parameters that define the per-GPU data allocations. See the normalization code.
- data.train_batch_size=32
    - prompts
- actor_rollout_ref.rollout.n=8
    - 每个 prompts sample 多少个 responses（grpo group size）
    - generation：train_batch_size * rollout_n
- actor.ppo_epochs=1
    - actor_rollout_ref.actor.ppo_mini_batch_size=16
        - train_batch_size // ppo_mini_batch_size => ppo training 多少次
    - actor_rollout_ref.actor.ppo_micro_batch_size_per_gpu=8
        - 这个是真正的 ppo training batch size
- forward-only (without grad)
    - actor_rollout_ref.**rollout**.log_prob_micro_batch_size_per_gpu=32
    - actor_rollout_ref.**ref**.log_prob_micro_batch_size_per_gpu=32 \

```
if not config.actor_rollout_ref.actor.use_dynamic_bsz:
    assert config.data.train_batch_size >= config.actor_rollout_ref.actor.ppo_mini_batch_size
    sp_size = config.actor_rollout_ref.actor.get('ulysses_sequence_parallel_size', 1)
    if config.actor_rollout_ref.actor.ppo_micro_batch_size is not None:
        assert config.actor_rollout_ref.actor.ppo_mini_batch_size % config.actor_rollout_ref.actor.ppo_micro_batch_size == 0
        assert config.actor_rollout_ref.actor.ppo_micro_batch_size * sp_size >= n_gpus

....


self.config.actor.ppo_mini_batch_size *= self.config.rollout.n
self.config.actor.ppo_mini_batch_size //= (self.device_mesh.size() // self.ulysses_sequence_parallel_size)
```

- ppo_mini_batch_size = 16 * 8 / 2 = 64
- ga = ppo_mini_batch_size / ppo_micro_batch_size_per_gpu = 64 / 8 = 8

- 一些限制
    - `config.data.train_batch_size >= config.actor_rollout_ref.actor.ppo_mini_batch_size`

### 其他

#### kl loss 

$$
\log\frac{\pi_\theta}{\pi_{ref}}=\log\pi_\theta - \log \pi_{ref}
$$
- kl_loss > 0: 表示当前策略 $ \pi_\theta$ 平均而言，对采样的响应序列 $a$ 分配了比参考策略 $ \pi_{ref}$ 更高的概率。这是 PPO 训练中期望看到的，因为策略正在学习**提高那些能带来高回报的序列的概率**。
- kl_loss < 0: 表示当前策略 $\pi_\theta$ 平均而言，对采样的响应序列 $a$ 分配了比参考策略 $ \pi_{ref} $ 更低的概率。这种情况可能在优化过程中短暂出现，或者如果参考策略本身就很擅长生成高回报序列。

#### entropy

- 最小0，最大 $\log|V|$
- 在 PPO 训练初期，策略可能还比较随机，熵会比较高。随着训练进行，策略变得更优化，熵可能会下降。entropy_coeff 的作用就是防止熵下降得过快或过低。

### 跑起来，跑得快

- actor_rollout_ref.model.use_remove_padding=True \
- fsdp
    - actor_rollout_ref.model.enable_gradient_checkpointing=True \
    - actor_rollout_ref.actor.fsdp_config.param_offload=False \
    - actor_rollout_ref.actor.fsdp_config.optimizer_offload=False \
- vllm
    - actor_rollout_ref.rollout.disable_log_stats=True